In [1]:
import os
import openai
from tqdm import tqdm
import json
import time
openai.api_key = ""  ## Note: an OpenAI api key needs to be added here
#openai.Model.list()

# The GPT-4 code based on the first set of 22 tweets was run in April 2023; The code based on two additional sets was run in September 2023

In [32]:
# A raw prompt created using 22 examples
recog_prompt = """This is a set of location description recognition problems.
The `Sentence` is a sentence containing location descriptions.
The goal is to infer which parts of the sentence represent location descriptions and the categories of the location descriptions. Split different location descriptions with `;`.
--

--
Sentence: Papa stranded in home. Water rising above waist. HELP 812 Wood Ln, 77828 #houstonflood
Q: Which parts of this sentence represent location descriptions?
A: C1: 812 Wood Ln, 77828 
--

--
Sentence: Anyone doing high water rescues in the Pasadena/Deer Park area? My daughter has been stranded in a parking lot all night
Q: Which parts of this sentence represent location descriptions?
A: C10: Pasadena/Deer Park
--

--
Sentence: Allen Parkway, Memorial, Waugh overpass, Spotts park and Buffalo Bayou park completely under water
Q: Which parts of this sentence represent location descriptions?
A: C2: Allen Parkway; C2: Memorial; C2: Waugh overpass; C7: Spotts park; C7: Buffalo Bayou park
--

--
Sentence: Streets Flooded: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd. HurricaneHarvey Houston
Q: Which parts of this sentence represent location descriptions?
A: C11: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd.; C9: Houston
--

--
Sentence: Cleaning supply drive is underway. 9-11 am today at Preston Hollow Presbyterian Church
Q: Which parts of this sentence represent location descriptions?
A: C8: Preston Hollow Presbyterian Church
--

--
Sentence: #Harvey LIVE from San Antonio, TX. Fatal car accident at Ingram Rd., Strong winds.
Q: Which parts of this sentence represent location descriptions?
A: C9: San Antonio; C9: TX; C2: Ingram Rd.
--

--
Sentence: 9:00AM update video from Hogan St over White Oak Bayou, I-10, I-45: water down about 4’ since last night.
Q: Which parts of this sentence represent location descriptions?
A: C5: Hogan St over White Oak Bayou; C3: I-10; C3: I-45
--

--
Sentence: Left Corpus bout to be in San Angelo #HurricaneHarvey Y’all be safe Avoided highway 37 Took the back road 
Q: Which parts of this sentence represent location descriptions?
A: C9: Corpus; C9: San Angelo; C3: highway 37
--

--
Sentence: Need trailers/trucks to move dogs from Park Location: Whites Park Pavillion off I-10 exit 61 Anahuac TX
Q: Which parts of this sentence represent location descriptions?
A: C7: Whites Park Pavillion; C3: I-10; C4: exit 61; C9: Anahuac; C9: TX
--

--
Sentence: Townsend exit, Sorters road and Hamblen road is flooded coming from 59 southbound HurricaneHarvery Harvey2017 
Q: Which parts of this sentence represent location descriptions?
A: C4: Townsend exit; C5: Sorters road and Hamblen road; C3: 59 southbound
--

--
Sentence: #Harvey does anyone know about the flooding conditions around Cypress Ridge High School?! #HurricaneHarvey
Q: Which parts of this sentence represent location descriptions?
A: C8: Cypress Ridge High School
--

--
Sentence: FYI to any of you in NW Houston/Lakewood Forest, Projections are showing Cypress Creek overflowing at Grant Rd
Q: Which parts of this sentence represent location descriptions?
A: C10: NW Houston/Lakewood Forest; C5: Cypress Creek overflowing at Grant Rd
--

--
Sentence: #HurricaneHarvey family needs rescuing at 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429, 2 elderly, one is 90 not steady in her feet
Q: Which parts of this sentence represent location descriptions?
A: C1: 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429
--

--
Sentence: Guys, this is I-45 @ Main Street in Houston. Crazy. #hurricane #harvey. . .
Q: Which parts of this sentence represent location descriptions?
A: C5: I-45 @ Main Street; C9: Houston
--

--
Sentence: Frontage Rd at the river #hurricaneHarvey #hurricaneharvey @ San Jacinto River
Q: Which parts of this sentence represent location descriptions?
A: C2: Frontage Rd; C6: San Jacinto River
--

--
Sentence: Pictures of downed trees and damaged apartment building on Airline Road in Corpus Christi.
Q: Which parts of this sentence represent location descriptions?
A: C2: Airline Road; C9: Corpus Christi
--

--
Sentence: Buffalo Bayou holding steady at 10,000 cfs at the gage near Terry Hershey Park
Q: Which parts of this sentence represent location descriptions?
A: C6: Buffalo Bayou; C7: Terry Hershey Park
--

--
Sentence: Major flooding at Clay Rd & Queenston in west Houston. Lots of rescues going on for ppl trapped...
Q: Which parts of this sentence represent location descriptions?
A: C5: Clay Rd & Queenston; C9: Houston
--

--
Sentence: HELP! A pregnant lady is stuck in her car on I-45 between Cypress Hill & Huffmeister exits #harvey
Q: Which parts of this sentence represent location descriptions?
A: C11: I-45 between Cypress Hill & Huffmeister exits
--

--
Sentence: If you need a place to escape #HurricaneHarvey, The Willie De Leon Civic Center: 300 E. Main St in Uvalde is open as a shelter
Q: Which parts of this sentence represent location descriptions?
A: C7: The Willie De Leon Civic Center; C1: 300 E. Main St in Uvalde
--

--
Sentence: Houston’s Buffalo Bayou Park - always among the first to flood. #Harvey
Q: Which parts of this sentence represent location descriptions?
A: C9: Houston; C7: Buffalo Bayou Park
--

--
Sentence: #HurricaneHarvey INTENSE eye wall of category 4 Hurricane Harvey from Rockport, TX
Q: Which parts of this sentence represent location descriptions?
A: C9: Rockport; C9: TX
--

--
Sentence: {TEXT}
Q: Which parts of this sentence represent location descriptions?
A:"""

In [2]:
# conversational prompt

conversational_recog_prompt = [{'role': 'system', 'content': "This is a set of location description recognition problems.\n The `Sentence` is a sentence containing location descriptions. The goal is to infer which parts of the sentence represent location descriptions and the categories of the location descriptions. Split different location descriptions with `;`."},
 
                {'role': 'user','content': 'Sentence: Papa stranded in home. Water rising above waist. HELP 812 Wood Ln, 77828 #houstonflood \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C1: 812 Wood Ln, 77828'},
 
                {'role': 'user','content': 'Sentence: Anyone doing high water rescues in the Pasadena/Deer Park area? My daughter has been stranded in a parking lot all night \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C10: Pasadena/Deer Park'},
                
                {'role': 'user','content': 'Sentence: Allen Parkway, Memorial, Waugh overpass, Spotts park and Buffalo Bayou park completely under water \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C2: Allen Parkway; C2: Memorial; C2: Waugh overpass; C7: Spotts park; C7: Buffalo Bayou park'},
               
                {'role': 'user','content': 'Sentence: Streets Flooded: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd. HurricaneHarvey Houston \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C11: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd.; C9: Houston'},
               
                {'role': 'user','content': 'Sentence: Cleaning supply drive is underway. 9-11 am today at Preston Hollow Presbyterian Church \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: Preston Hollow Presbyterian Church'},
               
                {'role': 'user','content': 'Sentence: #Harvey LIVE from San Antonio, TX. Fatal car accident at Ingram Rd., Strong winds. \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: San Antonio; C9: TX; C2: Ingram Rd.'},
                
                {'role': 'user','content': 'Sentence: 9:00AM update video from Hogan St over White Oak Bayou, I-10, I-45: water down about 4’ since last night. \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: Hogan St over White Oak Bayou; C3: I-10; C3: I-45'},
                
                {'role': 'user','content': 'Sentence: Left Corpus bout to be in San Angelo #HurricaneHarvey Y’all be safe Avoided highway 37 Took the back road \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Corpus; C9: San Angelo; C3: highway 37'},
               
                {'role': 'user','content': 'Sentence: Need trailers/trucks to move dogs from Park Location: Whites Park Pavillion off I-10 exit 61 Anahuac TX \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: Whites Park Pavillion; C3: I-10; C4: exit 61; C9: Anahuac; C9: TX'},
               
                {'role': 'user','content': 'Sentence: Townsend exit, Sorters road and Hamblen road is flooded coming from 59 southbound HurricaneHarvery Harvey2017 \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C4: Townsend exit; C5: Sorters road and Hamblen road; C3: 59 southbound'},
               
                {'role': 'user','content': 'Sentence: #Harvey does anyone know about the flooding conditions around Cypress Ridge High School?! #HurricaneHarvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: Cypress Ridge High School'},
                
                {'role': 'user','content': 'Sentence: FYI to any of you in NW Houston/Lakewood Forest, Projections are showing Cypress Creek overflowing at Grant Rd \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C10: NW Houston/Lakewood Forest; C5: Cypress Creek overflowing at Grant Rd'},
               
                {'role': 'user','content': 'Sentence: #HurricaneHarvey family needs rescuing at 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429, 2 elderly, one is 90 not steady in her feet \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C1: 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429'},
                
                {'role': 'user','content': 'Sentence: Guys, this is I-45 @ Main Street in Houston. Crazy. #hurricane #harvey. . . \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: I-45 @ Main Street; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: Frontage Rd at the river #hurricaneHarvey #hurricaneharvey @ San Jacinto River \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C2: Frontage Rd; C6: San Jacinto River'},
                
                {'role': 'user','content': 'Sentence: Pictures of downed trees and damaged apartment building on Airline Road in Corpus Christi. \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C2: Airline Road; C9: Corpus Christi'},
                
                {'role': 'user','content': 'Sentence: Buffalo Bayou holding steady at 10,000 cfs at the gage near Terry Hershey Park \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C6: Buffalo Bayou; C7: Terry Hershey Park'},
                
                {'role': 'user','content': 'Sentence: Major flooding at Clay Rd & Queenston in west Houston. Lots of rescues going on for ppl trapped... \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: Clay Rd & Queenston; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: HELP! A pregnant lady is stuck in her car on I-45 between Cypress Hill & Huffmeister exits #harvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C11: I-45 between Cypress Hill & Huffmeister exits'},
                
                {'role': 'user','content': 'Sentence: If you need a place to escape #HurricaneHarvey, The Willie De Leon Civic Center: 300 E. Main St in Uvalde is open as a shelter \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: The Willie De Leon Civic Center; C1: 300 E. Main St in Uvalde'},
                
                {'role': 'user','content': 'Sentence: Houston’s Buffalo Bayou Park - always among the first to flood. #Harvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Houston; C7: Buffalo Bayou Park'},
                
                {'role': 'user','content': 'Sentence: #HurricaneHarvey INTENSE eye wall of category 4 Hurricane Harvey from Rockport, TX \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Rockport; C9: TX'}
                 
               ]




In [2]:
# conversational prompt set2 selected from the data

conversational_recog_prompt_set2 = [{'role': 'system', 'content': "This is a set of location description recognition problems.\n The `Sentence` is a sentence containing location descriptions. The goal is to infer which parts of the sentence represent location descriptions and the categories of the location descriptions. Split different location descriptions with `;`."},
 
                {'role': 'user','content': 'Sentence: #Harvey Pls rescue 12 day baby family at 7 Woodview St, Houston, 77124, flooding will reach roof soon… \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C1: 7 Woodview St, Houston, 77124'},
 
                {'role': 'user','content': 'Sentence: What\'s left of my front and back yard in the League City/Dickinson area. \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C10: League City/Dickinson'},
                
                {'role': 'user','content': 'Sentence: Thanks Home Depot in Plano on Custer Rd for your help with Flood Cleanup Kits! \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Plano; C2: Custer Rd'},
               
                {'role': 'user','content': 'Sentence: A short video of the aftermath of HurricaneHarvey in Port Aransas on Cotter Ave. from Alister to Station St., 9/3/… \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Port Aransas; C11: Cotter Ave. from Alister to Station St.'},
               
                {'role': 'user','content': 'Sentence: First St Baptist Church needs water & cleaning supplies for Port Arthur community flooded by Harvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: First St Baptist Church; C9: Port Arthur'},
               
                {'role': 'user','content': 'Sentence: Can anyone get food to memorial Hermann hospital at Gessner Rd in Houston? Request from CajunNavy hurricaneHarvey … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: memorial Hermann hospital; C2: Gessner Rd; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: Garth road & IH10 at Baytown, Tx. Very High water. Baytown REPORT ON harvey2017 \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: Garth road & IH10; C9: Baytown; C9: Tx; C9: Baytown'},
                
                {'role': 'user','content': 'Sentence: someone placed a car here for ins purpose. parking lot of I-10 highway. Terry Hershey Park Parking lot. … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C3: I-10 highway; C7: Terry Hershey Park'},
               
                {'role': 'user','content': 'Sentence: Need trailers/trucks to move dogs from Park Location: Whites Park Pavillion off I-10 exit 61 Anahuac TX \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: Whites Park Pavillion; C3: I-10; C4: exit 61; C9: Anahuac; C9: TX'},
               
                {'role': 'user','content': 'Sentence: 59 north, towards the Polk St exit: Go to Convention Center. houstonflood KHOU11 288 texasflood \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C3: 59 north; C4: Polk St exit; C7: Convention Center'},
               
                {'role': 'user','content': 'Sentence: Community is responding at shelters in College Park High School and Magnolia High School in TheWoodlands Harvey … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: College Park High School; C8: Magnolia High School; C9: TheWoodlands'},
                
                {'role': 'user','content': 'Sentence: FYI to any of you in NW Houston/Lakewood Forest, Projections are showing Cypress Creek overflowing at Grant Rd \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C10: NW Houston/Lakewood Forest; C5: Cypress Creek overflowing at Grant Rd'},
               
                {'role': 'user','content': 'Sentence: Father in law in a wheelchair. Send help to 6312 Bapling Drive, Sugar Land, TX HarveyStorm HarveyRelief HARVEYHELP HoustonStrande \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C1: 6312 Bapling Drive, Sugar Land, TX'},
                
                {'role': 'user','content': 'Sentence: Gas is apparently going up this week in San Antonio because of HurricaneHarvey, it\'s still 2.51 at the co-op on 17th & 36th ave \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: San Antonio; C5: 17th & 36th ave'},
                
                {'role': 'user','content': 'Sentence: White Oak Bayou around Stude Park. HoustonFloods Houston Heights \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C6: White Oak Bayou; C7: Stude Park; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: Worried. A foot 2 go before Oyster Creek starts spilling into Lakes of Brightwater, Lakefront Drive, in front of our house … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C6: Oyster Creek; C6: Lakes of Brightwater; C2: Lakefront Drive'},
                
                {'role': 'user','content': 'Sentence: Texas City asking 18 homes on S. Humble Camp Rd. to evacuate in case GCWA reservoir breaches Harvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Texas City; C2: S. Humble Camp Rd.; C7: GCWA reservoir'},
                
                {'role': 'user','content': 'Sentence: Major flooding at Clay Rd & Queenston in west Houston. Lots of rescues going on for ppl trapped... \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: Clay Rd & Queenston; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: Closed due to train derailment in Harvey on Destrehan Ave between River Rd and 4th St traffic NOLA \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C11: Destrehan Ave between River Rd and 4th St'},
                
                {'role': 'user','content': 'Sentence: Drop Off Location: The Life Center, Charlotte, 2435 Toomey Ave. #SocksForHouston LCFellowship … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: The Life Center; C9: Charlotte; C1: 2435 Toomey Ave'},
                
                {'role': 'user','content': 'Sentence: Do we know how Minute Maid Park and NRG Park are?? Are they flooded as well as Honda Center? hurricaneharvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: Minute Maid Park; C7: NRG Park; C7: Honda Center'},
                
                {'role': 'user','content': 'Sentence: Eye wall still very much intact as Harvey is nearly stationary north of Rockport, TX. It\'s going to be a long nigh … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Rockport; C9: TX'}
                 
               ]



In [2]:
# conversational prompt based on synthesized data using geo-knowledge

conversational_recog_prompt_syn = [{'role': 'system', 'content': "This is a set of location description recognition problems.\n The `Sentence` is a sentence containing location descriptions. The goal is to infer which parts of the sentence represent location descriptions and the categories of the location descriptions. Split different location descriptions with `;`."},
 
                {'role': 'user','content': 'Sentence: Please help family needs rescue at 112 Wikleson Dr, Houston 42143 \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C1: 112 Wikleson Dr, Houston 42143'},
 
                {'role': 'user','content': 'Sentence: Heavy flooding in the Diego City/Amherst area. Please don\'t go there \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C10: Diego City/Amherst'},
                
                {'role': 'user','content': 'Sentence: Home Depot on Niagara Rd is donating tools for house repair #Harvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C2: Niagara Rd'},
               
                {'role': 'user','content': 'Sentence: There is an accident in Port Aransas on Bailey Rd. between Airport Dr and Station St \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Port Aransas; C11: Bailey Rd. between Airport Dr and Station St'},
               
                {'role': 'user','content': 'Sentence: Water & cleaning supplies are needed at Grace Family Bible Church. Please help out...  \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: Grace Family Bible Church'},
               
                {'role': 'user','content': 'Sentence: Help needed at memorial Hermann hospital at Gessner Rd in Houston! People need water and food there hurricaneHarvey … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: memorial Hermann hospital; C2: Gessner Rd; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: The intersection of Main road & I-60 is flooded. Very High water. harvey2017 \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: Main road & I-60'},
                
                {'role': 'user','content': 'Sentence: A person is trapped at the parking lot off I-10 highway near Buffalo Bayou park … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C3: I-10 highway; C7: Buffalo Bayou park'},
               
                {'role': 'user','content': 'Sentence: Exit 53 and exit 54 of I-11 are both flooded. Please avoid these two exits. Houston \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C4: Exit 53; C4: exit 54; C3: I-11; C9: Houston'},
               
                {'role': 'user','content': 'Sentence: 63 south, towards the Main St exit: Go to Convention Center. houstonflood 288 texasflood \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C3: 63 south; C4: Main St exit; C7: Convention Center'},
               
                {'role': 'user','content': 'Sentence: Shelters are provided in Williamsville High School. Please go there if you need a place to stay … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C8: Williamsville High School'},
                
                {'role': 'user','content': 'Sentence: For those of you living in NW Houston/Lakewood Forest, Grant Rd @ Cypress Rd is flooded. Avoid it! \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C10: NW Houston/Lakewood Forest; C5: Grant Rd @ Cypress Rd'},
               
                {'role': 'user','content': 'Sentence: Grandpa needs help at 1831 West Ridge Rd, Sugar Land, TX HarveyStorm HarveyRelief HARVEYHELP HoustonStrande \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C1: 1831 West Ridge Rd, Sugar Land, TX'},
                
                {'role': 'user','content': 'Sentence: High water at the 11th & 30th ave. Houston is battered by heavy rain \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: 11th & 30th ave; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: White Oak Bayou around Clear Lake Park. HoustonFloods Houston Heights \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C6: White Oak Bayou; C7: Clear Lake Park; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: This is Buffalo Creek running under Kingsview Bridge near Frontier Drive, crazy … \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C6: Buffalo Creek; C7: Kingsview Bridge; C2: Frontier Drive'},
                
                {'role': 'user','content': 'Sentence: Families living on S. Washington Rd. will have to evacuate before the water gets too high #Harvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C2: S. Washington Rd.'},
                
                {'role': 'user','content': 'Sentence: Major flooding at High Rd & 15 Ave. in west Houston. People trapped. Please help... \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C5: High Rd & 15 Ave.; C9: Houston'},
                
                {'role': 'user','content': 'Sentence: Highway 10 between River Rd and 4th St is flooded. Please use local roads NOLA \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C11: Highway 10 between River Rd and 4th St'},
                
                {'role': 'user','content': 'Sentence: If you have things to donate, you can drop them off at ClearField Community Center, Houston, 351 7th Ave. \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: ClearField Community Center; C9: Houston; C1: 351 7th Ave'},
                
                {'role': 'user','content': 'Sentence: Hurricane eye wall is approaching Port Aransas, TX. Pray...  \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C9: Port Aransas; C9: TX'},
                                
                {'role': 'user','content': 'Sentence: Do we know the flooding conditions at Buffalo Bayou Park and Minute Maid Park? Any information can help. hurricaneharvey \nQ: Which parts of this sentence represent location descriptions?'},
                {'role': 'assistant', 'content': 'C7: Buffalo Bayou Park; C7: Minute Maid Park'}
                   
               ]

In [3]:
# functions to load harvey dataset
def load_HarveyTweet2017_data(anno_file, data_dir, sep = "||"):
    with open(anno_file, 'r') as f:
        i = 0
        data_list = []
        for line in f:
            line = line.split("\n")[0]
            parts = line.split(sep)
            # print(parts)
            if i == 0:
                cols = parts
            else:
                file = os.path.join(data_dir, parts[0])
                text = parts[1]
                anno = json.loads(parts[2])
                item = {
                    "ID": i - 1,
                    cols[0]: file,
                    cols[1]: text,
                    cols[2]: anno,
                }

                data_list.append(item)

            i += 1
    return data_list


data_dir = "data/"
anno_file = os.path.join(data_dir, 'tweetTextAndAnnotation.txt')
data_list = load_HarveyTweet2017_data(anno_file, data_dir, sep = "||")



In [4]:
#GPT-4 code with 22 examples

n_tokens = 30
temp=0.2


# raw prompt version
def get_place_recognition_answer_logits(row, recog_prompt, text_key = "text"):
    
    text = row[text_key]
    
    recog_prompt_ = recog_prompt.replace("{TEXT}", text)
    
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": recog_prompt_}
      ],
      max_tokens =  n_tokens,
      temperature=temp,
      top_p=1,
        n=1,
        presence_penalty=0,
        frequency_penalty=0
    )

    answer = completion.choices[0].message["content"]
    
    res_dict = {'answer': answer}

    res_dict = {**res_dict,
                **dict(row)
               }
    return res_dict




# conversational version
def get_place_recognition_answer_logits_conversational(row, conv_recog_prompt, text_key = "text"):
    
    text = row[text_key]
    
    this_conv_prompt = conv_recog_prompt.copy()
    this_conv_prompt.append({'role': 'user','content': 'Sentence: '+text+' \nQ: Which parts of this sentence represent location descriptions? '})
    #print(this_conv_prompt)
    
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=this_conv_prompt,
      max_tokens =  n_tokens,
        temperature=temp,
        top_p=1,
        n=1,
        presence_penalty=0,
        frequency_penalty=0
    )

    answer = completion.choices[0].message["content"]
    
    res_dict = {'answer': answer}

    res_dict = {**res_dict,
                **dict(row)
               }
    return res_dict


In [ ]:
####### A test function for a single tweet
    
text = "someone placed a car here for ins purpose. parking lot of I-10 highway. Terry Hershey Park Parking lot. …"
#"#HurricaneHarvey INTENSE eye wall of category 4 Hurricane Harvey from Rockport, TX" 
#"A cattle drive takes to the streets of Dayton, Tx. due to Harvey txwx Video via Rebekah Brand" 
#"someone placed a car here for ins purpose. parking lot of I-10 highway. Terry Hershey Park Parking lot. …"



this_conv_prompt = conversational_recog_prompt_syn.copy()
this_conv_prompt.append({'role': 'user','content': 'Sentence: '+text+' \nQ: Which parts of this sentence represent location descriptions? '})
#print(this_conv_prompt)

completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=this_conv_prompt,
  max_tokens =  n_tokens,
    temperature=temp,
    top_p=1,
    n=1,
    presence_penalty=0,
    frequency_penalty=0
)

answer = completion.choices[0].message["content"]
    
answer

In [9]:
# set the path of the result file (res_file)
res_dir = "result/" 
if not os.path.exists(res_dir):
    os.mkdir(res_dir)

    
res_file = "{res_dir}/placerecognize_{engine}_22examples_syn_conversational.json".format(res_dir = res_dir, engine = "GPT-4")


# a function to read the result file
def read_res_file(res_file):
    res_dict_list = []
    
    if os.path.exists(res_file):
        with open(res_file, 'r') as f:
            for line in f:
                res_dict = json.loads(line)
                res_dict_list.append(res_dict)
    return res_dict_list

# retrieve only those tweets that haven't been processed to save cost
res_dict_list = read_res_file(res_file)
process_ids = [res_dict['ID'] for res_dict in res_dict_list]

cur_data_list = [item for item in data_list if item["ID"] not in process_ids]


In [10]:
len(cur_data_list)

1000

In [11]:
# Start to run the code
count = 0
for i, row in (pbar := tqdm(enumerate(cur_data_list), total= len(cur_data_list) )):
    
    #res_dict = get_place_recognition_answer_logits(row, recog_prompt,text_key = "tweetFulltext")
    
    res_dict = get_place_recognition_answer_logits_conversational(row, conversational_recog_prompt_syn, text_key = "tweetFulltext")
    
    with open(res_file, 'a') as f:
        f.write(json.dumps(res_dict) + "\n")
     
    
    #print the result for a preview
#     print("Tweet ID is: "+ res_dict['tweetFileID'])
#     print("Tweet text is: "+ res_dict['tweetFulltext'])
#     print("Annotation is: "+ str(res_dict['tweetAnnotation']))
#     print("Answer is: "+ res_dict['answer'])
    
    count += 1
    time.sleep(0.5)
    
#     if count == 2:
#         break

f.close()


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [36:56<00:00,  2.22s/it]


### Next, we use GPT-4 without any geo-knowledge

In [29]:
import re
conversational_recog_prompt_without_geo = conversational_recog_prompt = [{'role': 'user', 'content': "This is a set of location description recognition problems.\n The `Sentence` is a sentence containing location descriptions. The goal is to infer which parts of the sentence represent location descriptions and put them in double quotes."}]


In [40]:
# set the path of the result file (res_file)
res_dir = "result/" 
if not os.path.exists(res_dir):
    os.mkdir(res_dir)

    
res_file = "{res_dir}/placerecognize_{engine}_no_geo_conversational.json".format(res_dir = res_dir, engine = "GPT-4")


# a function to read the result file
def read_res_file(res_file):
    res_dict_list = []
    
    if os.path.exists(res_file):
        with open(res_file, 'r') as f:
            for line in f:
                res_dict = json.loads(line)
                res_dict_list.append(res_dict)
    return res_dict_list

# retrieve only those tweets that haven't been processed to save cost
res_dict_list = read_res_file(res_file)
process_ids = [res_dict['ID'] for res_dict in res_dict_list]

cur_data_list = [item for item in data_list if item["ID"] not in process_ids]
len(cur_data_list)

25

In [41]:
# Start to run the code without geo-knowledge
count = 0
for i, row in (pbar := tqdm(enumerate(cur_data_list), total= len(cur_data_list) )):
    
    #res_dict = get_place_recognition_answer_logits(row, recog_prompt,text_key = "tweetFulltext")
    
    res_dict = get_place_recognition_answer_logits_conversational(row, conversational_recog_prompt_without_geo, text_key = "tweetFulltext")
    
    original_answer = res_dict['answer']
    extracted_answer = re.findall('"([^"]*)"', original_answer)
    
    res_dict['origin_answer'] = res_dict['answer'] 
    res_dict['answer'] = ";".join(["CX: "+str(this_str) for this_str in extracted_answer])
    
    with open(res_file, 'a') as f:
        f.write(json.dumps(res_dict) + "\n")
     
    
    #print the result for a preview
#     print("Tweet ID is: "+ res_dict['tweetFileID'])
#     print("Tweet text is: "+ res_dict['tweetFulltext'])
#     print("Annotation is: "+ str(res_dict['tweetAnnotation']))
#     print("Original answer is: "+ res_dict['origin_answer'])
#     print("Answer is: "+ res_dict['answer'])
    
    count += 1
    time.sleep(0.5)
    
#     if count == 2:
#         break

f.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:25<00:00,  3.41s/it]
